In [9]:
# Librerías requeridas

import numpy as np
import pandas as pd
import datetime as dt
import gzip
import pickle
import re
import unidecode
from io import StringIO
import csv
import math as m

import fasttext
import tensorflow_hub as hub
import tensorflow_text
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn import FunctionSampler
from tempfile import mkdtemp
from shutil import rmtree

# **Funciones Principales**

In [13]:
#Función principal que ejecuta y desarrolla los experimentos 1 a 7

def base_model_iter_1(train, test, x, features, var, best_clfs, best_param, metricas_test, funcion_costo):
    
    # Generación del Descriptor

    train['descriptor'] = ''
    test['descriptor'] = ''

    for feature in features:
        
        train['descriptor'] = train['descriptor'] + ' ' + train[feature]
        train = train.dropna(subset=['descriptor'])
    
        test['descriptor'] = test['descriptor'] + ' ' + test[feature]
        test = test.dropna(subset=['descriptor'])
    
        var.remove(feature)

    train['descriptor'] = train['descriptor'].apply(lambda i: i.strip())
    test['descriptor'] = test['descriptor'].apply(lambda i: i.strip())

    print(train['descriptor'][0])

    print(feature.upper())
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    
    #Remove the feature that was used
    #var.remove(features[-1])
    print(var)

    #Ejecución del modelo FAST TEXT
    m = 0
    
    #Separación del conjunto de datos de entrenamiento inicial para el proceso de Fast Text

    train_ft, val_ft = train_test_split(train, test_size=0.9,stratify=train['gt'], shuffle=True, random_state=0)
    
    #Guardado de los conjunto de datos de entrenamiento y validación Fast Text en format txt
     
    train_ft[['label','descriptor']].to_csv(r'FT_Files/exp_{}'.format(x)+'_train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    
    val_ft[['label','descriptor']].to_csv(r'FT_Files/exp_{}'.format(x)+'_val.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    
    # Entrenamiento del modelo Fast Text
    model_sup = fasttext.train_supervised(input='FT_Files/exp_{}'.format(x)+'_train.txt', autotuneValidationFile='FT_Files/exp_{}'.format(x)+'_val.txt', autotuneDuration=duration, autotuneMetric='f1:__label__longer_70cm')

    #Ejecución de Métricas para Fast Text

    metricas_test = get_metricas(test.copy(),'test_FT_{}'.format(x), model_sup ,metricas_test, m, embed=None, model_emb=None, features_test=None, var=None)  
    
    #print('-------Metricas Test Inicial----')
    #print(metricas_test.info())
    #print(metricas_test.shape)
    #print('-------Metricas Test Inicial----')
    
    metricas_test, costo = costo_total(metricas_test,'costo_test_FT_{}'.format(x),'clase_test_FT_{}'.format(x))
    
    #print('-------Metricas Test Inicial MOD----')
    # #print(metricas_test.info())
    # #print(metricas_test.shape)
    # #print('-------Metricas Test Inicial MOD----')
    # #print('-------Funcion Costo Inicial----')
    # #print(funcion_costo.info())
    # #print(funcion_costo.shape)
    # #print('-------Funcion Costo Inicial----')
    
    funcion_costo = pd.concat([funcion_costo, costo],axis=1)
    
    #print('-------Funcion Costo Inicial MOD----')
    #print(funcion_costo.info())
    #print(funcion_costo.shape)
    #print('-------Funcion Costo Inicial MOD----')
        
    # Ejecución del resto de los modelos

    m = 1
    # #FEATURE TRANSFORMATION
    # #Insert 'descriptor' feature into list 'var' >> final training dataset features 
    var.insert(0,'descriptor') #OK
    print(var)
    features_test = var.copy() #OK to use in get_metrics()
    
    #Balanceo de clases por medio de Data Reduction

    rus = RandomUnderSampler(0.5, random_state=0)
    train_und, y_und = rus.fit_resample(train[var].copy(), train['gt'].copy())
    #print(train_und.info())
    #remove 'descriptor' in order to no modify original var
    var.remove('descriptor')
    print(var) 
    
    #ML - PIPE
    #Ejecución del pipeline de entrenamiento #FAST TEXT - SUPERVISADO
    
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'fts', metricas_test, funcion_costo,model_sup, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)
    
    #Ejecución del pipeline de entrenamiento FAST TEXT - NO SUPERVISADO
    
    train['descriptor'].to_csv(r'FT_Files/train_un_{}'.format(x) + '.txt', index=False, header=False)
    model_un = fasttext.train_unsupervised(input='FT_Files/train_un_{}'.format(x) + '.txt', lr=0.001, dim=300, epoch=50,minn=2,wordNgrams=3,loss='hs')
    
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'ftu', metricas_test, funcion_costo, model_un, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)

    #Ejecución del pipeline de entrenamiento TF - UNIVERSAL SENTENCE ENCODER
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'tf', metricas_test, funcion_costo, model_tf, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)
  

    #Ejecución del pipeline de entrenamiento SB - SBERT SENTENCE ENCODER
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'sb', metricas_test, funcion_costo, model_sb, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)
    
    print('Iteracion:'+str(x))
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    
    return metricas_test, funcion_costo, best_clfs, best_param

In [18]:
#Función principal que ejecuta y desarrolla los experimentos 8 a 13

def base_model_iter_2(train, test, x, feature, var, best_clfs, best_param, metricas_test, funcion_costo):
    
   # Generación del Descriptor
    
    train['descriptor'] = train['title']
    train['descriptor'] = train['descriptor'] + ' ' + train[feature]
    train = train.dropna(subset=['descriptor'])

    test['descriptor'] = test['title']
    test['descriptor'] = test['descriptor'] + ' ' + test[feature]
    test = test.dropna(subset=['descriptor'])

    print(feature.upper())
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    
    #Remove the feature that was used
    var.remove('title')
    var.remove(feature)
    print(var)
    
    # Ejecución del modelo FAST TEXT
    m = 0
    
    #Separación del conjunto de datos de entrenamiento inicial para el proceso de Fast Text

    train_ft, val_ft = train_test_split(train, test_size=0.9,stratify=train['gt'], shuffle=True, random_state=0)
    
    #Guardado de los conjunto de datos de entrenamiento y validación Fast Text en format txt

    train_ft[['label','descriptor']].to_csv(r'FT_Files/exp_{}'.format(x)+'_train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    
    val_ft[['label','descriptor']].to_csv(r'FT_Files/exp_{}'.format(x)+'_val.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    
    # Entrenamiento del modelo Fast Text

    model_sup = fasttext.train_supervised(input='FT_Files/exp_{}'.format(x)+'_train.txt', autotuneValidationFile='FT_Files/exp_{}'.format(x)+'_val.txt', autotuneDuration=duration, autotuneMetric='f1:__label__longer_70cm')

   #Ejecución de Métricas para Fast Text

    metricas_test = get_metricas(test.copy(),'test_FT_{}'.format(x), model_sup ,metricas_test, m, embed=None, model_emb=None, features_test=None, var=None)  
    
    #print('-------Metricas Test Inicial----')
    #print(metricas_test.info())
    #print(metricas_test.shape)
    #print('-------Metricas Test Inicial----')
    
    metricas_test, costo = costo_total(metricas_test,'costo_test_FT_{}'.format(x),'clase_test_FT_{}'.format(x))
    
    #print('-------Metricas Test Inicial MOD----')
    # #print(metricas_test.info())
    # #print(metricas_test.shape)
    # #print('-------Metricas Test Inicial MOD----')
    # #print('-------Funcion Costo Inicial----')
    # #print(funcion_costo.info())
    # #print(funcion_costo.shape)
    # #print('-------Funcion Costo Inicial----')
    
    funcion_costo = pd.concat([funcion_costo, costo],axis=1)
    
    #print('-------Funcion Costo Inicial MOD----')
    #print(funcion_costo.info())
    #print(funcion_costo.shape)
    #print('-------Funcion Costo Inicial MOD----')
        
    # Ejecución del resto de los modelos

    m = 1
    # #FEATURE TRANSFORMATION
    # #Insert 'descriptor' feature into list 'var' >> final training dataset features 
    var.insert(0,'descriptor') #OK
    print(var)
    features_test = var.copy() #OK to use in get_metrics()
    
    #Balanceo de clases por medio de Data Reduction

    rus = RandomUnderSampler(0.5, random_state=0)
    train_und, y_und = rus.fit_resample(train[var].copy(), train['gt'].copy())
    #print(train_und.info())
    #remove 'descriptor' in order to no modify original var
    var.remove('descriptor')
    print(var) 
    
    #ML - PIPE
    #Ejecución del pipeline de entrenamiento #FAST TEXT - SUPERVISADO
    
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'fts', metricas_test, funcion_costo,model_sup, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)
    
    #Ejecución del pipeline de entrenamiento FAST TEXT - NO SUPERVISADO
    
    train['descriptor'].to_csv(r'FT_Files/train_un_{}'.format(x) + '.txt', index=False, header=False)
    model_un = fasttext.train_unsupervised(input='FT_Files/train_un_{}'.format(x) + '.txt', lr=0.001, dim=300, epoch=50,minn=2,wordNgrams=3,loss='hs')
    
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'ftu', metricas_test, funcion_costo, model_un, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)

    #Ejecución del pipeline de entrenamiento TF - UNIVERSAL SENTENCE ENCODER
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'tf', metricas_test, funcion_costo, model_tf, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)
  

    #Ejecución del pipeline de entrenamiento SB - SBERT SENTENCE ENCODER
    metricas_test , funcion_costo, best, param = ml_pipe(train_und.copy(), y_und.copy(), 'sb', metricas_test, funcion_costo, model_sb, var, x, feature, features_test, test.copy())
    
    best_clfs.append(best)
    best_param.append(param)
    
    print('Iteracion:'+str(x))
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    
    return metricas_test, funcion_costo, best_clfs, best_param

# **Funciones Secundarias**

In [11]:
#Función que ejecuta el pipeline de entrenamiento para todos los experimentos realizados
#Esta función comprende la transformación del texto, Data Augmentation, Data Reduction y
#entrenamiento de los clasificadores de cada modelo. Se utiliza dentro de las funciones principales
#'base_model_iter_1' y 'base_model_iter_2'

def ml_pipe(train_res, y_res, embed, metricas_test, funcion_costo, model_emb, var, x, feature, features_test, test):

    #Embedding - 'descriptor
    #print('----Check Before Embedding----')
    #print(train_res.shape)
    print('---'+embed.upper()+'----')
    
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    
    #Transformación de la representación del descriptor
    train_res  = embedding(train_res, embed, model_emb)
    
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
  
    print('----Check After Embedding----')
    #print(train_res.shape)

    #Transformación a un objeto tipo array
    train_res = transform(train_res, var)
    print('----Check After Transform Array----')
    #print(train_res.shape)
    #print(train_res[0])

    #print('--------check-----') #OK

    
    #Valores y Variables del Espacio de Hiperparámetros
    #PCA
    n_features_options = list(range(5,20,5))
    
    #LR
    c_options = list(np.random.uniform(0,1,6)) + list(np.random.uniform(0,1000,4))
    penalty = ['l1','l2']
    
    #SVC
    kernel = ['rbf','sigmoid','poly','linear']

    #RFC
    n_estimators = [200,300,400, 500] 
    criterion = ['entropy', 'gini']
    min_samples_split = [0.01, 0.025, 0.05]
    min_samples_leaf = [0.1, 0.2, 0.3]
    class_weight = ['balanced', 'balanced_subsample']

    #GBC
    loss = ['deviance', 'exponential']
    learning_rate = [0.001, 0.005, 0.01, 0.03, 0.05]
    subsample = [0.2, 0.4, 0.6, 0.8, 1]
    n_estimators = [200,300,400, 500] 
    max_depth = [3,6,9,12]
    min_samples_split = [0.01, 0.025, 0.05]
    min_samples_leaf = [0.1, 0.2, 0.3]

    # Clasificadores
    models = [
                'LR', 
                'SVC',
                'RFC',
                'GBC'
            ]

    clfs = [
            LogisticRegression(random_state=0, solver='saga', max_iter=200, class_weight='balanced'),
            SVC(gamma='auto', random_state=0,probability=True, class_weight='balanced',max_iter=200),
            RandomForestClassifier(random_state=0),
            GradientBoostingClassifier(random_state=0, init='zero')
            ]

    #Espacio de hiperparámetros
    param_grid = { models[0]:{
                    'preprocessor__num__pca__n_components': n_features_options,
                    'classifier__C': c_options, 
                    'classifier__penalty': penalty
                    },
            
                    models[1]:{
                    'preprocessor__num__pca__n_components': n_features_options,
                    'classifier__C':c_options, 
                    'classifier__kernel':kernel
                      },

                    models[2]:{
                    'preprocessor__num__pca__n_components': n_features_options,
                    'classifier__n_estimators': n_estimators, 
                    'classifier__criterion':criterion,
                    'classifier__min_samples_split':min_samples_split,
                    'classifier__min_samples_leaf': min_samples_leaf,
                    'classifier__class_weight': class_weight
                    },
                
                    models[3]:{
                    'preprocessor__num__pca__n_components': n_features_options,
                    'classifier__loss': loss, 
                    'classifier__learning_rate': learning_rate,
                    'classifier__n_estimators': n_estimators,
                    'classifier__subsample': subsample,
                    'classifier__min_samples_split': min_samples_split,
                    'classifier__min_samples_leaf': min_samples_leaf,
                    'classifier__max_depth': max_depth 
                    }

                }

    cachedir = mkdtemp()

    numeric_features = list(range(train_res.shape[1] - len(var)))
    print('numeric features', numeric_features[-1])
    
    categorical_features = list(range(train_res.shape[1] - len(var) - 1, train_res.shape[1]))
    print( 'categorical features', categorical_features[-1])

    #Objeto para realizar Data Reduction
    under = RandomUnderSampler(sampling_strategy=1, random_state=0)

    #Construcción del pipeline de entrenamiento para variables numéricas
    numeric_transformer = Pipeline(steps=[
                                            ('scaler', MinMaxScaler()),
                                            ('pca', PCA(random_state=0))
                                        ])

    #Construcción del pipeline de entrenamiento para variables categóricas
    categorical_transformer = Pipeline(steps=[
                                                ('onehot', OneHotEncoder(categories='auto',handle_unknown='ignore'))
                                                ])

    #Unión de los pipelines de variables numéricas y categóricas
    preprocessor = ColumnTransformer(transformers=[
                                        ('num', numeric_transformer, numeric_features),
                                        ('cat', categorical_transformer, categorical_features)
                                          ])

    #Generación de datos sintéticos
    X_res = get_sintetic_data(train_res, y_res, categorical_features)
    print('Shape X_res ',X_res.shape)

    #Objeto función para insertar los datos sintéticos dentro del pipeline de entrenamiento
    add_sintetic_data = FunctionSampler(func=add_sintetic, validate=False, kw_args={'X_res': X_res})
    
    #Objeto de validación cruzada k-fold
    cv = StratifiedShuffleSplit(n_splits=10, train_size=0.1 ,test_size=0.05, random_state=0)

    #Ejecución del pipeline de entrenamiento para los diferentes clasificadores    
    for name, classifier in zip(models,clfs):
        
        print('-----Inicio CLF--------', name)
        now = datetime.datetime.now()
        print ("Current date and time : ")
        print (now.strftime("%Y-%m-%d %H:%M:%S"))

        print(name)

        clf = Pipeline(steps=[
                            ('add_sintetic_data', add_sintetic_data),
                            ('under', under),
                            ('preprocessor', preprocessor),
                            ('classifier',  classifier)
                                ], verbose=True, memory = cachedir)
        
        
        model = RandomizedSearchCV(clf, param_grid[name], cv=cv, n_iter=10, verbose=2 ,n_jobs=6)
        
        model.fit(train_res, y_res)
        
        print('-----Inicio Metrics--------')
        now = datetime.datetime.now()
        print ("Current date and time : ")
        print (now.strftime("%Y-%m-%d %H:%M:%S"))


        best = model.best_estimator_
        param = model.best_params_

        #Obtención de Métricas en base al conjunto Datos de Test

        #print('test_'+str(embed)+'_'+str(name)+'_{}'.format(x))
        #print('--------check Metricas-----')
        metricas = get_metricas(test,'test_'+str(embed)+'_'+str(name)+'_{}'.format(x), model, metricas_test, m, embed, model_emb, features_test, var)
        print('--------check Metricas Pipe -----')
        #print(metricas.info()) #OK
        #print('--------check Metricas Pipe ----') 
        #print('--------check Metricas-----')
        #print(metricas.head(1))
        #print('--------check Metrics-----')
        
        #print('costo_test_'+str(embed)+'_'+str(name)+'_{}'.format(x))
        #print('clase_test_'+str(embed)+'_'+str(name)+'_{}'.format(x))
        metricas_test, costo = costo_total(metricas,'costo_test_'+str(embed)+'_'+str(name)+'_{}'.format(x),'clase_test_'+str(embed)+'_'+str(name)+'_{}'.format(x))
        #print('--------check Metricas MOD Pipe-----')
        #print(metricas.info()) #OK
        #print('--------check Metricas MOD Pipe-----') 
        #print('--------check Costo Pipe-----')
        #print(costo.info()) #OK
        #print('--------check Costo Pipe-----') 
        #print(metricas.head(1))
        #print(costo.head(1))
        #print('--------check Costo-----')

        #metricas_test = pd.concat([metricas_test, metricas], axis=1)
        #print('--------check Metricas_Test Pipe-----')
        #print(metricas_test.info())
        #print('--------check Metricas_Test Pipe-----')
        funcion_costo = pd.concat([funcion_costo, costo],axis=1)
        #pred_test_tf_logisticRegression_1
        #print('--------check Funcion_Costo Pipe-----')
        #print(fui.info())
        #print('--------check Funcion_Costo Pipe-----')
        
        #print(metricas_test.head(1))
        #print(funcion_costo.head(1))
        
        print('-----Fin Metrics--------')
        now = datetime.datetime.now()
        print ("Current date and time : ")
        print (now.strftime("%Y-%m-%d %H:%M:%S"))

    rmtree(cachedir)

    return metricas_test , funcion_costo, best, param

In [7]:
#Función que contiene los diferentes métodos de transformación de las palabras. Esta función se ejecuta dentro de
#la función de entrenamiento 'ml_pipe'

def embedding(train_und, embed, model_emb):
    if embed == 'fts':
        train_und['descriptor'] = train_und['descriptor'].transform(lambda i: embed_sentence(i,model_emb)) # model_sup
    
    elif embed == 'ftu':
        train_und['descriptor'] = train_und['descriptor'].transform(lambda i: embed_sentence(i,model_emb)) #model_un
    
    elif embed == 'tf':
        train_und['descriptor'] = train_und['descriptor'].transform(lambda i: np.array(model_emb(i))) #model_tf
    
    else:
        train_und['descriptor'] = train_und['descriptor'].transform(lambda i: np.array(model_emb.encode(i))) #model_sb
    
    return train_und

In [6]:
#Función auxiliar de la función 'embedding' para obtener la representación de una sentencia.

def embed_sentence(sentence, model): #OK
  tokens = sentence.split()
  #print(len(tokens))
  vectors = [model.get_word_vector(i) for i in tokens]
  #print(len(vectors))
  return np.mean(vectors, axis=0)

In [5]:
#Función que realiza la transformación de las representaciones generadas a través de la función 'embedding'
#Es es necesario para que pueda seguir consumido por diferentes funciones del pipe de entrenamiento

def transform(data,features): #Funciona OK
    #print('--Inside Transform---')
    #print(data.shape)
    #count = 0
    #print(data.iloc[0])
    array = []
    for i in data.index:
        result = data['descriptor'][i].flatten().tolist()
        #print(type(result))
        for feature in features:
            result.append(data[feature][i])
        #count += 1
        
        array.append(result)
        #print(len(array))
        #break
    #print(count)
    array = np.array(array, dtype=object)
    #print('Array Transform')
    #print(array[0])
    #print(array.shape)
    return array

In [9]:
#Función que genera un cojunto de datos sintéticos
#Es utilizada durante el pipeline de entrenamiento, dentro la función 'ml_pipe'

def get_sintetic_data(df, y, categorical_features):
    
    categorical_features = [x+1 for x in categorical_features]
    
    df = pd.DataFrame(df)
    df.reset_index(drop=False, inplace=True)
    
    X_res, y_res = over_sampling(df, y, categorical_features)
    
    print('-----Over Sampling Done--------')
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    
    X_res = pd.DataFrame(X_res)
    X_res['y'] = y_res
    X_res = X_res[X_res.duplicated(subset=[0],keep='first')]
    X_res = X_res[X_res['y'] == 0]
    X_res.drop([0], axis=1, inplace=True)
    X_res.reset_index(drop=True, inplace=True)
    #print('NaNs X_res', X_res.isna().sum())

    return  X_res

In [8]:
#Función auxiliar a la función 'get_sintetic_data' que realiza un sobremuestreo y generación de datos sintéticos

def over_sampling(X, y, categorical_features):

    #print('Over Sampling Arrays')
    #print(X.shape)
    #print(y.shape)

    train_res = np.empty((0,X.shape[1]), int)
    y_res = np.empty((0,), int)

    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, stratify=y)
        #print("TRAIN:", len(train_index), "TEST:", len(test_index))
        smt = SMOTENC(random_state=0, sampling_strategy=0.8 ,n_jobs=6, categorical_features=categorical_features, k_neighbors=5)
        train_array, y_array = smt.fit_resample(X_train, y_train)
                
        train_res = np.vstack([train_res, train_array])
        y_res = np.concatenate((y_res, y_array))
        #print('over sampling done')
        #print(train_res.shape)
        #print(y_res.shape)
    return train_res, y_res

In [10]:
#Función que inserta los datos sintéticos dentro del pipeline de entrenamiento en la función 'ml_pipe'
 
def add_sintetic(X_train, y_train, X_res):
    
    X_train = pd.DataFrame(X_train)
    X_train['y'] = y_train
    
    X_res.columns = X_train.columns

    #print('NaNs X_train', X_train.isna().sum())
    #print(X_train.head(1))

    #print('shape before choice', X_res.shape)
    indexes = np.random.choice(len(X_res)-1, int(len(X_res)*0.3))
    X_res = X_res.iloc[indexes,:]
#   print('shape after choice', X_res.shape)
    #print('NaNs X_res_reducido', X_res.isna().sum())
    #print(X_res.head(1))
    
    df_final = pd.concat([X_train, X_res])
    #print('NaNs df_final', df_final.isna().sum())
    df_final.dropna(inplace=True)
    #print('NaNs df_final_drop', df_final.isna().sum())

    #print('shape df final', df_final.shape)
    df_final.reset_index(drop=True, inplace=True)
    #print(df_final.head(2))
    #df_final = df_final.infer_objects() 

    return df_final.drop(df_final.columns[-1], axis=1), df_final['y']

In [16]:
#Función que realiza el cálculo estimado del costo de envío para un usuario comprador.
#La columna generada 'costo_envio' se utiliza dentro de la función get_metricas para calcular
#el costo de cada registro dentro del conjunto de datos de evaluación

def costo_envio(peso):
  if peso < 0.5:
    return 48*0.19
  elif (peso > 0.5 and peso < 1):
    return 53*0.19
  elif (peso > 1 and peso < 2):
    return 62*0.19
  elif (peso > 2 and peso < 5):
    return 76*0.19
  elif (peso > 5 and peso < 9):
    return 96*0.19
  elif (peso > 9 and peso < 13):
    return 141*0.19
  elif (peso > 13 and peso < 17):
    return 188*0.19
  elif (peso > 17 and peso < 23):
    return 210*0.19
  elif (peso > 23 and peso < 29):
    return 220*0.19
  else:
    return 230*0.19

In [2]:
#Función que realiza el cálculo de métricas una vez que el pipeline de entrenamiento haya sido ejecutado dentro de la función de ml_pipe

def get_metricas(test,clase,model,metricas_test, m, embed, model_emb, features_test,var):
  print('---Metricas Running---')
  
  if m == 0: # Fast Text Supervised
    #print(clase)
    metricas_test['gt'] = test['gt'].astype('int').copy()
    #print(metricas_test.iloc[0])
    metricas_test['price'] = test['price'].copy()
    #print(metricas_test.iloc[0])
    metricas_test['costo_envio'] = test['costo_envio'].copy()
    #print(metricas_test.iloc[0])
    mapping = {('__label__shorter_70cm',):0 ,('__label__longer_70cm',):1}
    #print('pred_{}'.format(clase))
    metricas_test['pred_{}'.format(clase)] = test['descriptor'].transform(lambda i: model.predict(i)[0])
    #print(metricas_test.iloc[0])
  # label predict
    metricas_test['pred_{}'.format(clase)] = metricas_test['pred_{}'.format(clase)].map(mapping)
    #print(metricas_test.iloc[0])
  # map label str to (0,1)
    #print('prob_{}'.format(clase))
    metricas_test['prob_{}'.format(clase)] = metricas_test['gt'] != metricas_test['pred_{}'.format(clase)]
  # match gt / predict
    metricas_test['prob_{}'.format(clase)] = metricas_test['prob_{}'.format(clase)] * 1
  # transform to (0,1)
    metricas_test['prob_{}'.format(clase)] = metricas_test['prob_{}'.format(clase)] - test['descriptor'].transform(lambda i: model.predict(i)[1])
    #print(metricas_test.iloc[0])
  # 1 - prob or 0 - prob
    metricas_test['prob_{}'.format(clase)] = metricas_test['prob_{}'.format(clase)].abs().astype('float')
  
    metricas_test['prob_{}'.format(clase)] = metricas_test['prob_{}'.format(clase)].transform(lambda i: 1 if i > 1 else i)
    #print(metricas_test.iloc[0])
  else: # All model except Fast Text Supervised  INCLUIR LOS PASOS DE TRANSFORMACION SPARSA
    #print(features_test)
    test_array = test[features_test]
    #print('before embedding')
    #print(test_array.info())
    test_array = embedding(test_array, embed, model_emb)
    #print('After Embedding')
    #print(test_array.info())
    #print(test_array['descriptor'][0].shape)
    #print(var)

    test_array = transform(test_array, var)
    #print('After Transform')
    #print(test_array.shape)

    metricas_test['pred_{}'.format(clase)] =  model.predict(test_array)

    metricas_test = get_prob(metricas_test, test_array, clase, model)


  mapping = { '11':'TP','00':'TN','01':'FP','10':'FN'}

  x = 0
  thres = [0]
  for i in range(int(1/0.01)):
    x += 0.01
    thres.append(round(x,2))

  for t in thres:
                              
    metricas_test['clase_{}_'.format(clase)+str(t)] = (metricas_test['prob_{}'.format(clase)] > t) * 1

    metricas_test['clase_{}_'.format(clase)+str(t)] = metricas_test['gt'].astype('str') + metricas_test['clase_{}_'.format(clase)+str(t)].astype('str')

    metricas_test['clase_{}_'.format(clase)+str(t)] = metricas_test['clase_{}_'.format(clase)+str(t)].map(mapping)

    metricas_test['TP'] = (metricas_test['clase_{}_'.format(clase)+str(t)] == 'TP')*1
    metricas_test['FP'] = (metricas_test['clase_{}_'.format(clase)+str(t)] == 'FP')*1
    metricas_test['TN'] = (metricas_test['clase_{}_'.format(clase)+str(t)] == 'TN')*1
    metricas_test['FN'] = (metricas_test['clase_{}_'.format(clase)+str(t)] == 'FN')*1

    metricas_test['costo_{}_'.format(clase)+str(t)] = (- 15 + metricas_test['costo_envio'] * 0.12  * 0.15) * metricas_test['TN'] \
    + (15 - metricas_test['costo_envio'] * 0.12 * 0.15) * metricas_test['FP'] \
    + metricas_test['price'] * 0.19 * 0.13 * 0.15 * 0.6 * metricas_test['FN']  \
    - metricas_test['costo_envio'] * 0.12  * 0.15 * metricas_test['TP']        \
    
    metricas_test.drop(['TP','FP','TN','FN'],axis=1, inplace=True)

  return metricas_test

In [3]:
#Función utilizada dentro la de función 'get_metricas' para obtener las probabilidades de cada etiqueta predicha

def get_prob(metricas, test, clase, model):

    metricas['prob_{}'.format(clase)] = None

    metricas['prob_ml_0'] = pd.Series(model.predict_proba(test)[:,0])
    #print(metricas['prob_ml_0'][:2])
    metricas['prob_ml_1'] = pd.Series(model.predict_proba(test)[:,1])
    #print(metricas['prob_ml_1'][:2])
    #print('---Metricas Get Prob---')
    #print(metricas.info())
    #print(metricas.shape)
    #print('---Metricas Get Prob---')

    
    for ind in metricas.index:
    
        if metricas['gt'][ind] == 0:
            metricas['prob_{}'.format(clase)][ind] = metricas['prob_ml_0'][ind]
            #print(metricas['prob_{}'.format(clase)][ind])
        else:
            metricas['prob_{}'.format(clase)][ind] = metricas['prob_ml_1'][ind]
            #print(metricas['prob_{}'.format(clase)][ind])

    metricas.drop(['prob_ml_0','prob_ml_1'],axis=1, inplace=True)
    #print(metricas.info())
    return metricas

In [4]:
#Función que realiza el cálculo de costos / ahorros para cada punto del umbral de decisión de separación
#de clases de los clasificadores. Esta función se utiliza dentro de la fase del cálculo de métricas en el 
#pipeline de entrenamiento. Toma los valores calculados y el output de la función 'get_metricas'

def costo_total(metricas_test,itera,clase):
  columns = metricas_test.columns
  #print(columns)
  col_costos = [i for i in columns if itera in i]
  #print('--------Col Costo-----')
  #print(col_costos[:4])
  #print('--------Col Costo-----')
  col_clases = [i for i in columns if clase in i]
  #print('--------Col Clases-----')
  #print(col_clases[:4])
  #print('--------Col Clases-----')
  costos = []
  tp = []
  fp = []
  tn = []
  fn = []
  for column in col_costos:
    costos.append(metricas_test[column].sum())
  for column in col_clases:
    tp.append(metricas_test[metricas_test[column]=='TP'][column].count())
    fp.append(metricas_test[metricas_test[column]=='FP'][column].count())
    tn.append(metricas_test[metricas_test[column]=='TN'][column].count())
    fn.append(metricas_test[metricas_test[column]=='FN'][column].count())
  #print('Itera: ',itera)
  #print('Clase: ',clase)
  #print('Costos: ',len(costos))
  #print('TP: ',len(tp))
  #print('FP: ',len(fp))
  #print('TN: ',len(tn))
  #print('FN: ',len(fn))
  data = {itera : costos, clase + '_TP': tp, clase + '_FP': fp, clase + '_TN': tn, clase + '_FN': fn}
  data = pd.DataFrame(data)
  metricas_test.drop(col_costos,axis=1, inplace=True)
  metricas_test.drop(col_clases,axis=1, inplace=True)
  #print(metricas_test.info())
  return metricas_test , data

# **Ejecución Inicial**

In [12]:
#Modelo pre-entrednadode Tensor Flow
model_tf = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3") 

#Modelo pre-entrednadode BERT
model_sb = SentenceTransformer('distiluse-base-multilingual-cased')

#Tiempo Autotune - Fast Text 
duration = 600

INFO:absl:Using /var/folders/c8/492nb22j1tn4_d8z9nfc7p340000gq/T/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3: 140.03MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3: 260.03MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3, Total size: 334.32MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'.


In [10]:
#Carga inicial del conjunto de datos preprocesado

ds = pd.read_pickle('ds_pre.pkl')

In [12]:
#Cálculo estimado de costos de envío

ds['costo_envio'] = ds['shp_weight'].copy()
ds['costo_envio'] = ds['costo_envio'] / 1000
ds['costo_envio'] = ds['costo_envio'].transform(lambda i: costo_envio(i))

In [13]:
#Separación de los datos en conjuntos de entrenamiento y test

train, test = train_test_split(ds, test_size=0.15, stratify=ds['gt'], shuffle=True, random_state=0)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1669841 entries, 1841928 to 1921610
Data columns (total 13 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   gt           1669841 non-null  int64  
 1   title        1669841 non-null  object 
 2   label        1669841 non-null  object 
 3   domain_id    1669841 non-null  object 
 4   category_id  1669841 non-null  object 
 5   seller_id    1669841 non-null  object 
 6   brand_name   1669841 non-null  object 
 7   L1           1669841 non-null  object 
 8   L2           1669841 non-null  object 
 9   L3           1669841 non-null  object 
 10  shp_weight   1669841 non-null  float64
 11  price        1669841 non-null  float64
 12  costo_envio  1669841 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 178.4+ MB


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294678 entries, 1887334 to 244463
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   gt           294678 non-null  int64  
 1   title        294678 non-null  object 
 2   label        294678 non-null  object 
 3   domain_id    294678 non-null  object 
 4   category_id  294678 non-null  object 
 5   seller_id    294678 non-null  object 
 6   brand_name   294678 non-null  object 
 7   L1           294678 non-null  object 
 8   L2           294678 non-null  object 
 9   L3           294678 non-null  object 
 10  shp_weight   294678 non-null  float64
 11  price        294678 non-null  float64
 12  costo_envio  294678 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 31.5+ MB


In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

# **Experimentos 1 a 7**

In [15]:
#Experimento 1

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title']
x = 1

In [16]:
metricas_test_1, funcion_costo_1, best_clfs_1, best_param_1 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli
TITLE
Current date and time : 
2020-12-29 15:27:29
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2020-12-29 15:40:52
Current date and time : 
2020-12-29 15:41:09
----Check After Embedding----
----Check After Transform Array----
numeric features 216
categorical features 223
-----Over Sampling Done--------
Current date and time : 
2020-12-29 15:47:26
Shape X_res  (36406, 225)
-----Inicio CLF-------- LR
Current date and time : 
2020-12-29 15:47:40
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  4.9min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed: 

In [17]:
funcion_costo_1.to_pickle(r'funcion_costo_full_1.pkl')
metricas_test_1.to_pickle(r'metricas_test_full_1.pkl')

f = gzip.open('best_clfs_1.pklz','wb')
pickle.dump(best_clfs_1,f)
f.close()

f = gzip.open('best_param_1.pklz','wb')
pickle.dump(best_param_1,f)
f.close()

In [19]:
metricas_test_1.head()

,gt,price,costo_envio,pred_test_FT_1,prob_test_FT_1,pred_test_fts_LR_1,prob_test_fts_LR_1,pred_test_fts_SVC_1,prob_test_fts_SVC_1,pred_test_fts_RFC_1,...,pred_test_tf_GBC_1,prob_test_tf_GBC_1,pred_test_sb_LR_1,prob_test_sb_LR_1,pred_test_sb_SVC_1,prob_test_sb_SVC_1,pred_test_sb_RFC_1,prob_test_sb_RFC_1,pred_test_sb_GBC_1,prob_test_sb_GBC_1
0,1,120.90,14.44,0,0.003379,1,0.939661,1,0.60183,1,...,0,0.376146,1,0.963674,0,0.416034,0,0.499982,0,0.499894
1,1,270.00,11.78,0,0.010879,1,0.896657,1,0.544625,1,...,1,0.6783,1,0.895678,0,0.341839,1,0.500736,0,0.499894
2,1,200.33,14.44,0,0.026333,1,0.887253,1,0.518558,1,...,1,0.786211,1,0.935574,0,0.354428,1,0.500885,0,0.499894
3,1,49.90,9.12,0,0.003024,1,0.898818,1,0.604543,1,...,1,0.570882,1,0.923446,0,0.288192,1,0.500665,0,0.499894
4,1,69.90,9.12,0,0.001309,1,0.946685,1,0.71863,1,...,1,0.5586,1,0.89213,0,0.27987,1,0.500336,0,0.499894


In [22]:
funcion_costo_1.head()

,clase_test_FT_1_FN,clase_test_FT_1_FP,clase_test_FT_1_TN,clase_test_FT_1_TP,clase_test_fts_GBC_1_FN,clase_test_fts_GBC_1_FP,clase_test_fts_GBC_1_TN,clase_test_fts_GBC_1_TP,clase_test_fts_LR_1_FN,clase_test_fts_LR_1_FP,...,costo_test_ftu_RFC_1,costo_test_ftu_SVC_1,costo_test_sb_GBC_1,costo_test_sb_LR_1,costo_test_sb_RFC_1,costo_test_sb_SVC_1,costo_test_tf_GBC_1,costo_test_tf_LR_1,costo_test_tf_RFC_1,costo_test_tf_SVC_1
0,32,6433,11,193875,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,142688,5166,1278,51219,0,6444,0,193907,140,5155,...,25582.487985,25582.487985,25582.487985,-17202.962168,25582.487985,25582.487985,25582.487985,-4843.524651,25582.487985,25582.487985
2,161038,4737,1707,32869,0,6444,0,193907,305,4560,...,25582.487985,25582.487985,25582.487985,-33049.398227,25582.487985,25582.487985,25582.487985,-20576.227069,25582.487985,25582.487985
3,169708,4446,1998,24199,0,6444,0,193907,468,4178,...,25582.487985,25582.487985,25582.487985,-42890.416376,25582.487985,25582.487985,25582.487985,-30737.017042,25582.487985,25582.487985
4,174980,4223,2221,18927,0,6444,0,193907,645,3894,...,25582.487985,25582.487985,25582.487985,-50775.269809,25582.487985,25582.487985,25582.487985,-39320.561649,25582.487985,25582.487985


In [19]:
#Experimento 2

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title','domain_id']
x = 2

In [20]:
metricas_test_2, funcion_costo_2, best_clfs_2, best_param_2 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli revivalismo
DOMAIN_ID
Current date and time : 
2020-12-30 13:03:08
['category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
['category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2020-12-30 13:15:12
Current date and time : 
2020-12-30 13:15:22
----Check After Embedding----
----Check After Transform Array----
numeric features 181
categorical features 187
-----Over Sampling Done--------
Current date and time : 
2020-12-30 13:19:44
Shape X_res  (36369, 189)
-----Inicio CLF-------- LR
Current date and time : 
2020-12-30 13:19:56
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  7.1min finished
[Pipel

In [21]:
funcion_costo_2.to_pickle(r'funcion_costo_full_2.pkl')
metricas_test_2.to_pickle(r'metricas_test_full_2.pkl')

In [22]:
f = gzip.open('best_clfs_2.pklz','wb')
pickle.dump(best_clfs_2,f)
f.close()

f = gzip.open('best_param_2.pklz','wb')
pickle.dump(best_param_2,f)
f.close()

In [24]:
metricas_test_2.head()

,gt,price,costo_envio,pred_test_FT_2,prob_test_FT_2,pred_test_fts_LR_2,prob_test_fts_LR_2,pred_test_fts_SVC_2,prob_test_fts_SVC_2,pred_test_fts_RFC_2,...,pred_test_tf_GBC_2,prob_test_tf_GBC_2,pred_test_sb_LR_2,prob_test_sb_LR_2,pred_test_sb_SVC_2,prob_test_sb_SVC_2,pred_test_sb_RFC_2,prob_test_sb_RFC_2,pred_test_sb_GBC_2,prob_test_sb_GBC_2
0,1,120.90,14.44,0,0.000010,1,0.991732,0,0.39424,1,...,0,0.37261,1,0.998879,0,0.468205,0,0.499758,0,0.376585
1,1,270.00,11.78,0,0.004023,1,0.947277,0,0.507058,1,...,1,0.638448,1,0.982424,0,0.441445,0,0.499767,1,0.6545
2,1,200.33,14.44,0,0.012172,1,0.971271,0,0.5217,0,...,1,0.682269,1,0.99404,1,0.5,0,0.499758,1,0.661534
3,1,49.90,9.12,0,0.000817,1,0.991804,0,0.505308,1,...,1,0.647959,1,0.998304,0,0.477094,1,0.500265,1,0.578049
4,1,69.90,9.12,0,0.000012,1,0.986273,0,0.53179,1,...,1,0.638866,1,0.98519,1,0.509588,0,0.499794,1,0.625613


In [26]:
funcion_costo_2.head()

,clase_test_FT_2_FN,clase_test_FT_2_FP,clase_test_FT_2_TN,clase_test_FT_2_TP,clase_test_fts_GBC_2_FN,clase_test_fts_GBC_2_FP,clase_test_fts_GBC_2_TN,clase_test_fts_GBC_2_TP,clase_test_fts_LR_2_FN,clase_test_fts_LR_2_FP,...,costo_test_ftu_RFC_2,costo_test_ftu_SVC_2,costo_test_sb_GBC_2,costo_test_sb_LR_2,costo_test_sb_RFC_2,costo_test_sb_SVC_2,costo_test_tf_GBC_2,costo_test_tf_LR_2,costo_test_tf_RFC_2,costo_test_tf_SVC_2
0,1,6249,195,193906,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,115,4306,2138,193792,0,6444,0,193907,0,3591,...,25582.487985,25582.487985,25582.487985,-85350.082275,25582.487985,25582.487985,25582.487985,16927.576570,25582.487985,25582.487985
2,168,3916,2528,193739,0,6444,0,193907,0,2999,...,25582.487985,25582.487985,25582.487985,-97794.182595,25582.487985,25582.487985,25582.487985,2196.119421,25582.487985,25582.487985
3,204,3704,2740,193703,0,6444,0,193907,0,2632,...,25582.487985,25582.487985,25582.487985,-105920.466075,25582.487985,25582.487985,25582.487985,-9037.488615,25582.487985,25582.487985
4,237,3538,2906,193670,0,6444,0,193907,0,2388,...,25582.487985,25582.487985,25582.487985,-110900.655585,25582.487985,25582.487985,25582.487985,-17707.865402,25582.487985,25582.487985


In [23]:
#Experimento 3

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title','domain_id','category_id']
x = 3

In [24]:
metricas_test_3, funcion_costo_3, best_clfs_3, best_param_3 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli revivalismo tecnico
CATEGORY_ID
Current date and time : 
2020-12-31 08:59:53
['seller_id', 'brand_name', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
['seller_id', 'brand_name', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2020-12-31 09:12:14
Current date and time : 
2020-12-31 09:12:26
----Check After Embedding----
----Check After Transform Array----
numeric features 164
categorical features 169
-----Over Sampling Done--------
Current date and time : 
2020-12-31 09:16:24
Shape X_res  (36490, 171)
-----Inicio CLF-------- LR
Current date and time : 
2020-12-31 09:16:36
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  2.0min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  6.4min finished
[Pipeline] . (step 1 of 4) Processing add

In [25]:
funcion_costo_3.to_pickle(r'funcion_costo_full_3.pkl')
metricas_test_3.to_pickle(r'metricas_test_full_3.pkl')

f = gzip.open('best_clfs_3.pklz','wb')
pickle.dump(best_clfs_3,f)
f.close()

f = gzip.open('best_param_3.pklz','wb')
pickle.dump(best_param_3,f)
f.close()

In [28]:
metricas_test_3.head()

,gt,price,costo_envio,pred_test_FT_3,prob_test_FT_3,pred_test_fts_LR_3,prob_test_fts_LR_3,pred_test_fts_SVC_3,prob_test_fts_SVC_3,pred_test_fts_RFC_3,...,pred_test_tf_GBC_3,prob_test_tf_GBC_3,pred_test_sb_LR_3,prob_test_sb_LR_3,pred_test_sb_SVC_3,prob_test_sb_SVC_3,pred_test_sb_RFC_3,prob_test_sb_RFC_3,pred_test_sb_GBC_3,prob_test_sb_GBC_3
0,1,120.90,14.44,0,0.000010,1,0.872088,1,0.551747,1,...,0,0.423532,1,0.694473,0,0.485055,1,0.500719,1,0.50009
1,1,270.00,11.78,0,0.000311,1,0.862953,1,0.737775,1,...,1,0.551893,1,0.805103,0,0.466355,1,0.500611,1,0.50009
2,1,200.33,14.44,0,0.000067,1,0.912803,1,0.509911,1,...,1,0.589763,1,0.907252,0,0.411211,1,0.500844,1,0.50009
3,1,49.90,9.12,0,0.000008,1,0.934505,1,0.688601,1,...,1,0.585606,1,0.909678,0,0.490095,1,0.501277,1,0.50009
4,1,69.90,9.12,0,0.000010,1,0.931516,1,0.719911,1,...,1,0.560471,0,0.497537,0,0.398951,1,0.500899,1,0.50009


In [29]:
funcion_costo_3.head()

,clase_test_FT_3_FN,clase_test_FT_3_FP,clase_test_FT_3_TN,clase_test_FT_3_TP,clase_test_fts_GBC_3_FN,clase_test_fts_GBC_3_FP,clase_test_fts_GBC_3_TN,clase_test_fts_GBC_3_TP,clase_test_fts_LR_3_FN,clase_test_fts_LR_3_FP,...,costo_test_ftu_RFC_3,costo_test_ftu_SVC_3,costo_test_sb_GBC_3,costo_test_sb_LR_3,costo_test_sb_RFC_3,costo_test_sb_SVC_3,costo_test_tf_GBC_3,costo_test_tf_LR_3,costo_test_tf_RFC_3,costo_test_tf_SVC_3
0,185,5178,1266,193722,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,190047,3343,3101,3860,0,6444,0,193907,140,5343,...,25582.487985,25582.487985,25582.487985,15281.396083,25582.487985,25582.487985,25582.487985,12204.106986,25582.487985,25582.487985
2,190902,3177,3267,3005,0,6444,0,193907,288,4698,...,25582.487985,25582.487985,25582.487985,1357.814219,25582.487985,25582.487985,25582.487985,-3177.010134,25582.487985,25582.487985
3,191331,3074,3370,2576,0,6444,0,193907,460,4289,...,25582.487985,25582.487985,25582.487985,-10486.130335,25582.487985,25582.487985,25582.487985,-14396.233747,25582.487985,25582.487985
4,191611,2996,3448,2296,0,6444,0,193907,661,3974,...,25582.487985,25582.487985,25582.487985,-19224.032479,25582.487985,25582.487985,25582.487985,-24010.571807,25582.487985,25582.487985


In [26]:
#Experimento 4

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title','domain_id','category_id', 'seller_id']
x = 4

In [27]:
metricas_test_4, funcion_costo_4, best_clfs_4, best_param_4 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli revivalismo tecnico cdedgjcda
SELLER_ID
Current date and time : 
2021-01-01 05:20:51
['brand_name', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'brand_name', 'L1', 'L2', 'L3']
['brand_name', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2021-01-01 05:32:52
Current date and time : 
2021-01-01 05:33:03
----Check After Embedding----
----Check After Transform Array----
numeric features 99
categorical features 103
-----Over Sampling Done--------
Current date and time : 
2021-01-01 05:35:35
Shape X_res  (36480, 105)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-01 05:35:41
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  4.0min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=   7.9s
[P

In [28]:
funcion_costo_4.to_pickle(r'funcion_costo_full_4.pkl')
metricas_test_4.to_pickle(r'metricas_test_full_4.pkl')

f = gzip.open('best_clfs_4.pklz','wb')
pickle.dump(best_clfs_4,f)
f.close()

f = gzip.open('best_param_4.pklz','wb')
pickle.dump(best_param_4,f)
f.close()

In [31]:
metricas_test_4.head()

,gt,price,costo_envio,pred_test_FT_4,prob_test_FT_4,pred_test_fts_LR_4,prob_test_fts_LR_4,pred_test_fts_SVC_4,prob_test_fts_SVC_4,pred_test_fts_RFC_4,...,pred_test_tf_GBC_4,prob_test_tf_GBC_4,pred_test_sb_LR_4,prob_test_sb_LR_4,pred_test_sb_SVC_4,prob_test_sb_SVC_4,pred_test_sb_RFC_4,prob_test_sb_RFC_4,pred_test_sb_GBC_4,prob_test_sb_GBC_4
0,1,120.90,14.44,0,0.000022,1,0.990459,1,0.5,0,...,0,0.499933,0,0.401085,1,0.521982,1,0.500004,0,0.454831
1,1,270.00,11.78,0,0.005906,1,0.785965,1,0.5,0,...,0,0.499933,1,0.882041,0,0.491808,1,0.500216,1,0.7179
2,1,200.33,14.44,0,0.012506,1,0.942603,1,0.5,1,...,0,0.499933,1,0.975598,0,0.493604,1,0.500522,1,0.67303
3,1,49.90,9.12,0,0.005910,1,0.961701,1,0.5,1,...,0,0.499933,1,0.985458,1,0.510909,1,0.500108,1,0.621954
4,1,69.90,9.12,0,0.000062,1,0.963852,1,0.5,1,...,0,0.499933,0,0.128403,0,0.5,1,0.50037,1,0.72883


In [32]:
funcion_costo_4.head()

,clase_test_FT_4_FN,clase_test_FT_4_FP,clase_test_FT_4_TN,clase_test_FT_4_TP,clase_test_fts_GBC_4_FN,clase_test_fts_GBC_4_FP,clase_test_fts_GBC_4_TN,clase_test_fts_GBC_4_TP,clase_test_fts_LR_4_FN,clase_test_fts_LR_4_FP,...,costo_test_ftu_RFC_4,costo_test_ftu_SVC_4,costo_test_sb_GBC_4,costo_test_sb_LR_4,costo_test_sb_RFC_4,costo_test_sb_SVC_4,costo_test_tf_GBC_4,costo_test_tf_LR_4,costo_test_tf_RFC_4,costo_test_tf_SVC_4
0,50,6431,13,193857,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,153075,5352,1092,40832,0,6444,0,193907,976,3700,...,25582.487985,25582.487985,25582.487985,-37331.958255,25582.487985,25582.487985,25582.487985,-34507.853964,25582.487985,25582.487985
2,167945,5012,1432,25962,0,6444,0,193907,1569,3230,...,25582.487985,25582.487985,25582.487985,-49315.159416,25582.487985,25582.487985,25582.487985,-45126.604649,25582.487985,25582.487985
3,174529,4830,1614,19378,0,6444,0,193907,2050,2977,...,25582.487985,25582.487985,25582.487985,-56758.613413,25582.487985,25582.487985,25582.487985,-53616.053963,25582.487985,25582.487985
4,178527,4649,1795,15380,2038,4446,1998,191869,2548,2806,...,25582.487985,25582.487985,25582.487985,-61370.701479,25582.487985,25582.487985,25582.487985,-59158.904922,25582.487985,25582.487985


In [29]:
#Experimento 5

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title','domain_id','category_id', 'seller_id', 'brand_name']
x = 5

In [30]:
metricas_test_5, funcion_costo_5, best_clfs_5, best_param_5 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli revivalismo tecnico cdedgjcda tork polimet
BRAND_NAME
Current date and time : 
2021-01-02 05:29:21
['L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'L1', 'L2', 'L3']
['L1', 'L2', 'L3']
---FTS----
Current date and time : 
2021-01-02 05:42:02
Current date and time : 
2021-01-02 05:42:14
----Check After Embedding----
----Check After Transform Array----
numeric features 99
categorical features 102
-----Over Sampling Done--------
Current date and time : 
2021-01-02 05:44:23
Shape X_res  (36427, 104)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-02 05:44:28
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  4.1min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=   7.9s
[Pipeline] ............. (step

In [31]:
funcion_costo_5.to_pickle(r'funcion_costo_full_5.pkl')
metricas_test_5.to_pickle(r'metricas_test_full_5.pkl')

f = gzip.open('best_clfs_5.pklz','wb')
pickle.dump(best_clfs_5,f)
f.close()

f = gzip.open('best_param_5.pklz','wb')
pickle.dump(best_param_5,f)
f.close()

In [34]:
metricas_test_5.head()

,gt,price,costo_envio,pred_test_FT_5,prob_test_FT_5,pred_test_fts_LR_5,prob_test_fts_LR_5,pred_test_fts_SVC_5,prob_test_fts_SVC_5,pred_test_fts_RFC_5,...,pred_test_tf_GBC_5,prob_test_tf_GBC_5,pred_test_sb_LR_5,prob_test_sb_LR_5,pred_test_sb_SVC_5,prob_test_sb_SVC_5,pred_test_sb_RFC_5,prob_test_sb_RFC_5,pred_test_sb_GBC_5,prob_test_sb_GBC_5
0,1,120.90,14.44,0,0.000037,1,0.941188,1,0.5,1,...,0,0.332985,0,0.0707171,0,0.514774,0,0.499403,0,0.496273
1,1,270.00,11.78,0,0.008720,1,0.715692,1,0.5,1,...,1,0.679401,1,0.904238,0,0.515941,1,0.500389,1,0.563337
2,1,200.33,14.44,0,0.009670,1,0.765737,1,0.5,1,...,1,0.800347,1,0.988126,0,0.517251,1,0.500604,1,0.561685
3,1,49.90,9.12,0,0.001872,1,0.873735,1,0.5,1,...,1,0.639599,1,0.965614,0,0.5,1,0.500552,1,0.512131
4,1,69.90,9.12,0,0.000063,1,0.921571,1,0.5,1,...,1,0.528982,0,0.202147,0,0.509682,1,0.500604,1,0.563098


In [35]:
funcion_costo_5.head()

,clase_test_FT_5_FN,clase_test_FT_5_FP,clase_test_FT_5_TN,clase_test_FT_5_TP,clase_test_fts_GBC_5_FN,clase_test_fts_GBC_5_FP,clase_test_fts_GBC_5_TN,clase_test_fts_GBC_5_TP,clase_test_fts_LR_5_FN,clase_test_fts_LR_5_FP,...,costo_test_ftu_RFC_5,costo_test_ftu_SVC_5,costo_test_sb_GBC_5,costo_test_sb_LR_5,costo_test_sb_RFC_5,costo_test_sb_SVC_5,costo_test_tf_GBC_5,costo_test_tf_LR_5,costo_test_tf_RFC_5,costo_test_tf_SVC_5
0,54,6430,14,193853,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,154021,5284,1160,39886,0,6444,0,193907,112,5444,...,25582.487985,25582.487985,25582.487985,-648.361425,25582.487985,25582.487985,25582.487985,2615.911394,25582.487985,25582.487985
2,168396,4982,1462,25511,0,6444,0,193907,273,4817,...,25582.487985,25582.487985,25582.487985,-26382.502028,25582.487985,25582.487985,25582.487985,-14316.091446,25582.487985,25582.487985
3,174834,4805,1639,19073,215,5073,1371,193692,440,4447,...,25582.487985,25582.487985,25582.487985,-30474.141551,25582.487985,25582.487985,25582.487985,-25293.767786,25582.487985,25582.487985
4,178630,4654,1790,15277,406,4671,1773,193501,606,4159,...,25582.487985,25582.487985,25582.487985,-33715.062843,25582.487985,25582.487985,25582.487985,-31981.401165,25582.487985,25582.487985


In [32]:
# Experimento 6

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1']
x = 6

In [33]:
metricas_test_6, funcion_costo_6, best_clfs_6, best_param_6 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli revivalismo tecnico cdedgjcda tork polimet acessorios para veiculos
L1
Current date and time : 
2021-01-03 09:26:37
['L2', 'L3']
---Metricas Running---
['descriptor', 'L2', 'L3']
['L2', 'L3']
---FTS----
Current date and time : 
2021-01-03 09:40:05
Current date and time : 
2021-01-03 09:40:27
----Check After Embedding----
----Check After Transform Array----
numeric features 302
categorical features 304
-----Over Sampling Done--------
Current date and time : 
2021-01-03 09:45:23
Shape X_res  (36442, 306)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-03 09:45:47
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  4.9min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed: 16.4min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=  30.6s
[Pipeline] ............. (step

In [34]:
funcion_costo_6.to_pickle(r'funcion_costo_full_6.pkl')
metricas_test_6.to_pickle(r'metricas_test_full_6.pkl')

f = gzip.open('best_clfs_6.pklz','wb')
pickle.dump(best_clfs_6,f)
f.close()

f = gzip.open('best_param_6.pklz','wb')
pickle.dump(best_param_6,f)
f.close()

In [37]:
metricas_test_6.head()

,gt,price,costo_envio,pred_test_FT_6,prob_test_FT_6,pred_test_fts_LR_6,prob_test_fts_LR_6,pred_test_fts_SVC_6,prob_test_fts_SVC_6,pred_test_fts_RFC_6,...,pred_test_tf_GBC_6,prob_test_tf_GBC_6,pred_test_sb_LR_6,prob_test_sb_LR_6,pred_test_sb_SVC_6,prob_test_sb_SVC_6,pred_test_sb_RFC_6,prob_test_sb_RFC_6,pred_test_sb_GBC_6,prob_test_sb_GBC_6
0,1,120.90,14.44,0,0.000169,1,0.838754,0,0.749907,1,...,0,0.499702,0,0.300229,1,0.527608,0,0.499659,0,0.337533
1,1,270.00,11.78,0,0.001561,1,0.742252,1,0.842634,1,...,0,0.499702,1,0.849246,0,0.466097,0,0.499704,0,0.499266
2,1,200.33,14.44,0,0.002542,1,0.815506,1,0.824553,1,...,0,0.499702,1,0.920165,0,0.464565,1,0.50049,1,0.609201
3,1,49.90,9.12,0,0.001419,1,0.861523,1,0.856762,1,...,0,0.499702,1,0.840911,0,0.450528,1,0.500039,1,0.529
4,1,69.90,9.12,0,0.000903,1,0.95474,1,0.929402,1,...,0,0.499702,0,0.365335,0,0.459783,1,0.500039,1,0.538528


In [38]:
funcion_costo_6.head()

,clase_test_FT_6_FN,clase_test_FT_6_FP,clase_test_FT_6_TN,clase_test_FT_6_TP,clase_test_fts_GBC_6_FN,clase_test_fts_GBC_6_FP,clase_test_fts_GBC_6_TN,clase_test_fts_GBC_6_TP,clase_test_fts_LR_6_FN,clase_test_fts_LR_6_FP,...,costo_test_ftu_RFC_6,costo_test_ftu_SVC_6,costo_test_sb_GBC_6,costo_test_sb_LR_6,costo_test_sb_RFC_6,costo_test_sb_SVC_6,costo_test_tf_GBC_6,costo_test_tf_LR_6,costo_test_tf_RFC_6,costo_test_tf_SVC_6
0,0,6440,4,193907,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,20,5067,1377,193887,0,6444,0,193907,67,5801,...,25582.487985,25582.487985,25582.487985,20410.178942,25582.487985,25582.487985,25582.487985,24470.706910,25582.487985,25582.487985
2,45,4523,1921,193862,0,6444,0,193907,166,5322,...,25582.487985,25582.487985,25582.487985,15721.210523,25582.487985,25582.487985,25582.487985,21234.647603,25582.487985,25582.487985
3,64,4168,2276,193843,0,6444,0,193907,270,4930,...,25582.487985,25582.487985,25582.487985,9902.019972,25582.487985,25582.487985,25582.487985,18728.180144,25582.487985,25582.487985
4,81,3882,2562,193826,0,6444,0,193907,398,4617,...,25582.487985,25582.487985,25582.487985,1904.738745,25582.487985,25582.487985,25582.487985,16380.984817,25582.487985,25582.487985


In [15]:
#Experimento 7

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
features = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2']
x = 7

In [16]:
metricas_test_7, funcion_costo_7, best_clfs_7, best_param_7 = base_model_iter_1(train.copy(), test.copy(), x, features, var, best_clfs, best_param, metricas_test, funcion_costo)

bau tork litros bagageiro titan fan preto poli revivalismo tecnico cdedgjcda tork polimet acessorios para veiculos aces motos quadriciclos
L2
Current date and time : 
2021-01-05 22:09:24
['L3']
---Metricas Running---
['descriptor', 'L3']
['L3']
---FTS----
Current date and time : 
2021-01-05 22:21:24
Current date and time : 
2021-01-05 22:21:41
----Check After Embedding----
----Check After Transform Array----
numeric features 85
categorical features 86
-----Over Sampling Done--------
Current date and time : 
2021-01-05 22:23:32
Shape X_res  (36396, 88)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-05 22:23:36
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  3.5min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=   5.8s
[Pipeline] ............. (s

In [17]:
funcion_costo_7.to_pickle(r'funcion_costo_full_7.pkl')
metricas_test_7.to_pickle(r'metricas_test_full_7.pkl')

f = gzip.open('best_clfs_7.pklz','wb')
pickle.dump(best_clfs_7,f)
f.close()

f = gzip.open('best_param_7.pklz','wb')
pickle.dump(best_param_7,f)
f.close()

In [40]:
metricas_test_7.head()

,gt,price,costo_envio,pred_test_FT_7,prob_test_FT_7,pred_test_fts_LR_7,prob_test_fts_LR_7,pred_test_fts_SVC_7,prob_test_fts_SVC_7,pred_test_fts_RFC_7,...,pred_test_tf_GBC_7,prob_test_tf_GBC_7,pred_test_sb_LR_7,prob_test_sb_LR_7,pred_test_sb_SVC_7,prob_test_sb_SVC_7,pred_test_sb_RFC_7,prob_test_sb_RFC_7,pred_test_sb_GBC_7,prob_test_sb_GBC_7
0,1,120.90,14.44,0,0.000009,1,0.985878,1,0.712374,1,...,0,0.499766,0,0.29876,1,0.489816,0,0.499893,1,0.500008
1,1,270.00,11.78,0,0.010110,1,0.920223,1,0.552537,1,...,0,0.499766,1,0.740221,0,0.479539,0,0.499893,1,0.500008
2,1,200.33,14.44,0,0.003989,1,0.911104,1,0.557535,1,...,0,0.499766,1,0.911903,0,0.475781,0,0.499994,1,0.500008
3,1,49.90,9.12,0,0.000201,1,0.967443,1,0.605493,1,...,0,0.499766,1,0.80773,0,0.488256,1,0.500128,1,0.500008
4,1,69.90,9.12,0,0.000007,1,0.996494,1,0.75118,1,...,0,0.499766,0,0.447675,0,0.483912,0,0.499994,1,0.500008


In [41]:
funcion_costo_7.head()

,clase_test_FT_7_FN,clase_test_FT_7_FP,clase_test_FT_7_TN,clase_test_FT_7_TP,clase_test_fts_GBC_7_FN,clase_test_fts_GBC_7_FP,clase_test_fts_GBC_7_TN,clase_test_fts_GBC_7_TP,clase_test_fts_LR_7_FN,clase_test_fts_LR_7_FP,...,costo_test_ftu_RFC_7,costo_test_ftu_SVC_7,costo_test_sb_GBC_7,costo_test_sb_LR_7,costo_test_sb_RFC_7,costo_test_sb_SVC_7,costo_test_tf_GBC_7,costo_test_tf_LR_7,costo_test_tf_RFC_7,costo_test_tf_SVC_7
0,201,6351,93,193706,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,179004,4657,1787,14903,0,6444,0,193907,702,4241,...,25582.487985,25582.487985,25582.487985,24113.806372,25582.487985,25582.487985,25582.487985,24720.845357,25582.487985,25582.487985
2,184306,4359,2085,9601,0,6444,0,193907,1263,3784,...,25582.487985,25582.487985,25582.487985,20735.873555,25582.487985,25582.487985,25582.487985,21072.852990,25582.487985,25582.487985
3,186521,4202,2242,7386,0,6444,0,193907,1812,3510,...,25582.487985,25582.487985,25582.487985,17838.487117,25582.487985,25582.487985,25582.487985,18743.909244,25582.487985,25582.487985
4,187721,4094,2350,6186,0,6444,0,193907,2323,3305,...,25582.487985,25582.487985,25582.487985,15276.496881,25582.487985,25582.487985,25582.487985,16370.338074,25582.487985,25582.487985


# **Experimentos 8 a 13**

In [22]:
# Experimento 8

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

feature =  'category_id'
var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
x = 8

In [23]:
metricas_test_8, funcion_costo_8, best_clfs_8, best_param_8 = base_model_iter_2(train.copy(), test.copy(), x, feature, var, best_clfs, best_param, metricas_test, funcion_costo)

CATEGORY_ID
Current date and time : 
2021-01-09 02:34:02
['domain_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'domain_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
['domain_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2021-01-09 02:46:03
Current date and time : 
2021-01-09 02:46:13
----Check After Embedding----
----Check After Transform Array----
numeric features 183
categorical features 189
-----Over Sampling Done--------
Current date and time : 
2021-01-09 02:50:37
Shape X_res  (36610, 191)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-09 02:50:50
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  6.9min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=  16.6

In [24]:
funcion_costo_8.to_pickle(r'funcion_costo_full_8.pkl')
metricas_test_8.to_pickle(r'metricas_test_full_8.pkl')

f = gzip.open('best_clfs_8.pklz','wb')
pickle.dump(best_clfs_8,f)
f.close()

f = gzip.open('best_param_8.pklz','wb')
pickle.dump(best_param_8,f)
f.close()

In [43]:
metricas_test_8.head()

,gt,price,costo_envio,pred_test_FT_8,prob_test_FT_8,pred_test_fts_LR_8,prob_test_fts_LR_8,pred_test_fts_SVC_8,prob_test_fts_SVC_8,pred_test_fts_RFC_8,...,pred_test_tf_GBC_8,prob_test_tf_GBC_8,pred_test_sb_LR_8,prob_test_sb_LR_8,pred_test_sb_SVC_8,prob_test_sb_SVC_8,pred_test_sb_RFC_8,prob_test_sb_RFC_8,pred_test_sb_GBC_8,prob_test_sb_GBC_8
0,1,120.90,14.44,0,0.000010,1,0.991169,1,0.565952,1,...,0,0.499872,1,0.998945,1,0.50901,0,0.499292,0,0.46646
1,1,270.00,11.78,0,0.004066,1,0.941921,0,0.44703,1,...,0,0.499872,1,0.985002,0,0.481122,1,0.500345,1,0.521117
2,1,200.33,14.44,0,0.014307,1,0.964524,0,0.410408,1,...,0,0.499872,1,0.995732,0,0.5,1,0.500536,1,0.521117
3,1,49.90,9.12,0,0.000405,1,0.991367,0,0.5,1,...,0,0.499872,1,0.998681,0,0.429508,1,0.500465,1,0.521117
4,1,69.90,9.12,0,0.000018,1,0.987686,1,0.548256,1,...,0,0.499872,1,0.984518,1,0.53478,1,0.500536,1,0.521117


In [44]:
funcion_costo_8.head()

,clase_test_FT_8_FN,clase_test_FT_8_FP,clase_test_FT_8_TN,clase_test_FT_8_TP,clase_test_fts_GBC_8_FN,clase_test_fts_GBC_8_FP,clase_test_fts_GBC_8_TN,clase_test_fts_GBC_8_TP,clase_test_fts_LR_8_FN,clase_test_fts_LR_8_FP,...,costo_test_ftu_RFC_8,costo_test_ftu_SVC_8,costo_test_sb_GBC_8,costo_test_sb_LR_8,costo_test_sb_RFC_8,costo_test_sb_SVC_8,costo_test_tf_GBC_8,costo_test_tf_LR_8,costo_test_tf_RFC_8,costo_test_tf_SVC_8
0,183,6237,207,193724,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,176220,4298,2146,17687,0,6444,0,193907,147,5154,...,25582.487985,25582.487985,25582.487985,2220.458941,25582.487985,25582.487985,25582.487985,-78452.449416,25582.487985,25582.487985
2,182009,3988,2456,11898,0,6444,0,193907,306,4582,...,25582.487985,25582.487985,25582.487985,-14603.424707,25582.487985,25582.487985,25582.487985,-89315.738797,25582.487985,25582.487985
3,184538,3834,2610,9369,0,6444,0,193907,465,4199,...,25582.487985,25582.487985,25582.487985,-24780.757439,25582.487985,25582.487985,25582.487985,-94151.609644,25582.487985,25582.487985
4,186036,3726,2718,7871,0,6444,0,193907,642,3926,...,25582.487985,25582.487985,25582.487985,-33957.957183,25582.487985,25582.487985,25582.487985,-95443.163013,25582.487985,25582.487985


In [25]:
# Experimento 9

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

feature =  'seller_id'
var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
x = 9 

In [26]:
metricas_test_9, funcion_costo_9, best_clfs_9, best_param_9 = base_model_iter_2(train.copy(), test.copy(), x, feature, var, best_clfs, best_param, metricas_test, funcion_costo)

SELLER_ID
Current date and time : 
2021-01-10 08:32:53
['domain_id', 'category_id', 'brand_name', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'domain_id', 'category_id', 'brand_name', 'L1', 'L2', 'L3']
['domain_id', 'category_id', 'brand_name', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2021-01-10 08:45:20
Current date and time : 
2021-01-10 08:45:36
----Check After Embedding----
----Check After Transform Array----
numeric features 302
categorical features 308
-----Over Sampling Done--------
Current date and time : 
2021-01-10 08:51:10
Shape X_res  (36521, 310)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-10 08:51:33
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed: 15.2min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=  

In [27]:
funcion_costo_9.to_pickle(r'funcion_costo_full_9.pkl')
metricas_test_9.to_pickle(r'metricas_test_full_9.pkl')

f = gzip.open('best_clfs_9.pklz','wb')
pickle.dump(best_clfs_9,f)
f.close()

f = gzip.open('best_param_9.pklz','wb')
pickle.dump(best_param_9,f)
f.close()

In [46]:
metricas_test_9.head()

,gt,price,costo_envio,pred_test_FT_9,prob_test_FT_9,pred_test_fts_LR_9,prob_test_fts_LR_9,pred_test_fts_SVC_9,prob_test_fts_SVC_9,pred_test_fts_RFC_9,...,pred_test_tf_GBC_9,prob_test_tf_GBC_9,pred_test_sb_LR_9,prob_test_sb_LR_9,pred_test_sb_SVC_9,prob_test_sb_SVC_9,pred_test_sb_RFC_9,prob_test_sb_RFC_9,pred_test_sb_GBC_9,prob_test_sb_GBC_9
0,1,120.90,14.44,0,0.000010,1,0.95097,1,0.51889,1,...,0,0.499871,1,0.947761,0,0.465898,1,0.50067,0,0.384464
1,1,270.00,11.78,0,0.006603,1,0.948635,0,0.296776,1,...,0,0.499871,1,0.951038,0,0.410155,1,0.501258,1,0.593029
2,1,200.33,14.44,0,0.007713,1,0.898293,0,0.291942,0,...,0,0.499871,1,0.909475,0,0.417896,1,0.500376,1,0.578046
3,1,49.90,9.12,0,0.000359,1,0.939447,0,0.289049,1,...,0,0.499871,1,0.916847,0,0.383372,0,0.499701,1,0.572568
4,1,69.90,9.12,0,0.000009,1,0.89189,0,0.277196,1,...,0,0.499871,1,0.585282,0,0.457533,1,0.500934,1,0.578351


In [47]:
funcion_costo_9.head()

,clase_test_FT_9_FN,clase_test_FT_9_FP,clase_test_FT_9_TN,clase_test_FT_9_TP,clase_test_fts_GBC_9_FN,clase_test_fts_GBC_9_FP,clase_test_fts_GBC_9_TN,clase_test_fts_GBC_9_TP,clase_test_fts_LR_9_FN,clase_test_fts_LR_9_FP,...,costo_test_ftu_RFC_9,costo_test_ftu_SVC_9,costo_test_sb_GBC_9,costo_test_sb_LR_9,costo_test_sb_RFC_9,costo_test_sb_SVC_9,costo_test_tf_GBC_9,costo_test_tf_LR_9,costo_test_tf_RFC_9,costo_test_tf_SVC_9
0,59,6418,26,193848,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,157395,5249,1195,36512,0,6444,0,193907,548,4314,...,25582.487985,25582.487985,25582.487985,-59886.638926,25582.487985,25582.487985,25582.487985,11437.715116,25582.487985,25582.487985
2,169829,4938,1506,24078,0,6444,0,193907,969,3757,...,25582.487985,25582.487985,25582.487985,-69359.862232,25582.487985,25582.487985,25582.487985,-4192.318947,25582.487985,25582.487985
3,175370,4751,1693,18537,0,6444,0,193907,1357,3450,...,25582.487985,25582.487985,25582.487985,-76139.905073,25582.487985,25582.487985,25582.487985,-13815.295629,25582.487985,25582.487985
4,178623,4587,1857,15284,0,6444,0,193907,1722,3219,...,25582.487985,25582.487985,25582.487985,-80802.527296,25582.487985,25582.487985,25582.487985,-22118.379190,25582.487985,25582.487985


In [28]:
# Experimento 10

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

feature = 'brand_name'
var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
x = 10 

In [29]:
metricas_test_10, funcion_costo_10, best_clfs_10, best_param_10 = base_model_iter_2(train.copy(), test.copy(), x, feature, var, best_clfs, best_param, metricas_test, funcion_costo)

BRAND_NAME
Current date and time : 
2021-01-11 22:03:07
['domain_id', 'category_id', 'seller_id', 'L1', 'L2', 'L3']
---Metricas Running---
['descriptor', 'domain_id', 'category_id', 'seller_id', 'L1', 'L2', 'L3']
['domain_id', 'category_id', 'seller_id', 'L1', 'L2', 'L3']
---FTS----
Current date and time : 
2021-01-11 22:15:44
Current date and time : 
2021-01-11 22:15:57
----Check After Embedding----
----Check After Transform Array----
numeric features 123
categorical features 129
-----Over Sampling Done--------
Current date and time : 
2021-01-11 22:19:14
Shape X_res  (36466, 131)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-11 22:19:22
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.5min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  5.0min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_data, total=  10

In [30]:
funcion_costo_10.to_pickle(r'funcion_costo_full_10.pkl')
metricas_test_10.to_pickle(r'metricas_test_full_10.pkl')

f = gzip.open('best_clfs_10.pklz','wb')
pickle.dump(best_clfs_10,f)
f.close()

f = gzip.open('best_param_10.pklz','wb')
pickle.dump(best_param_10,f)
f.close()

In [49]:
metricas_test_10.head()

,gt,price,costo_envio,pred_test_FT_10,prob_test_FT_10,pred_test_fts_LR_10,prob_test_fts_LR_10,pred_test_fts_SVC_10,prob_test_fts_SVC_10,pred_test_fts_RFC_10,...,pred_test_tf_GBC_10,prob_test_tf_GBC_10,pred_test_sb_LR_10,prob_test_sb_LR_10,pred_test_sb_SVC_10,prob_test_sb_SVC_10,pred_test_sb_RFC_10,prob_test_sb_RFC_10,pred_test_sb_GBC_10,prob_test_sb_GBC_10
0,1,120.90,14.44,0,0.000319,1,0.979751,1,0.970745,0,...,0,0.382611,1,0.997043,0,0.473514,0,0.499968,0,0.395169
1,1,270.00,11.78,0,0.003118,1,0.89968,0,0.600946,1,...,1,0.610864,1,0.971748,0,0.3599,1,0.500534,1,0.734433
2,1,200.33,14.44,0,0.005382,1,0.925903,1,0.831102,1,...,1,0.618253,1,0.972313,0,0.410146,1,0.500025,1,0.65814
3,1,49.90,9.12,0,0.000323,1,0.968461,1,0.896217,1,...,1,0.617817,1,0.988602,0,0.423854,1,0.500165,1,0.616259
4,1,69.90,9.12,0,0.001110,1,0.985486,1,0.989125,1,...,1,0.570873,1,0.967135,0,0.433997,0,0.499927,1,0.715117


In [50]:
funcion_costo_10.head()

,clase_test_FT_10_FN,clase_test_FT_10_FP,clase_test_FT_10_TN,clase_test_FT_10_TP,clase_test_fts_GBC_10_FN,clase_test_fts_GBC_10_FP,clase_test_fts_GBC_10_TN,clase_test_fts_GBC_10_TP,clase_test_fts_LR_10_FN,clase_test_fts_LR_10_FP,...,costo_test_ftu_RFC_10,costo_test_ftu_SVC_10,costo_test_sb_GBC_10,costo_test_sb_LR_10,costo_test_sb_RFC_10,costo_test_sb_SVC_10,costo_test_tf_GBC_10,costo_test_tf_LR_10,costo_test_tf_RFC_10,costo_test_tf_SVC_10
0,7,6441,3,193900,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,132622,5390,1054,61285,0,6444,0,193907,1031,3550,...,25582.487985,25582.487985,25582.487985,14069.311503,25582.487985,25582.487985,25582.487985,629.412182,25582.487985,25582.487985
2,154927,4995,1449,38980,0,6444,0,193907,1710,3016,...,25582.487985,25582.487985,25582.487985,2341.546110,25582.487985,25582.487985,25582.487985,-16649.558944,25582.487985,25582.487985
3,165147,4710,1734,28760,0,6444,0,193907,2307,2708,...,25582.487985,25582.487985,25582.487985,-6391.719939,25582.487985,25582.487985,25582.487985,-28679.105693,25582.487985,25582.487985
4,171445,4493,1951,22462,0,6444,0,193907,2832,2519,...,25582.487985,25582.487985,25582.487985,-13378.632183,25582.487985,25582.487985,25582.487985,-35595.888710,25582.487985,25582.487985


In [31]:
# Experimento 11

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

feature =  'L1'
var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
x = 11

In [32]:
metricas_test_11, funcion_costo_11, best_clfs_11, best_param_11 = base_model_iter_2(train.copy(), test.copy(), x, feature, var, best_clfs, best_param, metricas_test, funcion_costo)

L1
Current date and time : 
2021-01-13 07:19:11
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L2', 'L3']
---Metricas Running---
['descriptor', 'domain_id', 'category_id', 'seller_id', 'brand_name', 'L2', 'L3']
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L2', 'L3']
---FTS----
Current date and time : 
2021-01-13 07:31:26
Current date and time : 
2021-01-13 07:31:37
----Check After Embedding----
----Check After Transform Array----
numeric features 217
categorical features 223
-----Over Sampling Done--------
Current date and time : 
2021-01-13 07:36:48
Shape X_res  (36508, 225)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-13 07:37:04
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  9.1min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_

In [33]:
funcion_costo_11.to_pickle(r'funcion_costo_full_11.pkl')
metricas_test_11.to_pickle(r'metricas_test_full_11.pkl')

f = gzip.open('best_clfs_11.pklz','wb')
pickle.dump(best_clfs_11,f)
f.close()

f = gzip.open('best_param_11.pklz','wb')
pickle.dump(best_param_11,f)
f.close()

In [52]:
metricas_test_11.head()

,gt,price,costo_envio,pred_test_FT_11,prob_test_FT_11,pred_test_fts_LR_11,prob_test_fts_LR_11,pred_test_fts_SVC_11,prob_test_fts_SVC_11,pred_test_fts_RFC_11,...,pred_test_tf_GBC_11,prob_test_tf_GBC_11,pred_test_sb_LR_11,prob_test_sb_LR_11,pred_test_sb_SVC_11,prob_test_sb_SVC_11,pred_test_sb_RFC_11,prob_test_sb_RFC_11,pred_test_sb_GBC_11,prob_test_sb_GBC_11
0,1,120.90,14.44,0,0.006885,1,0.994159,1,0.84159,0,...,0,0.383925,1,0.918147,1,0.5,0,0.499765,0,0.499831
1,1,270.00,11.78,0,0.015607,1,0.99305,1,0.801498,0,...,1,0.541196,1,0.745678,1,0.5,1,0.500467,0,0.499831
2,1,200.33,14.44,0,0.006320,1,0.981473,1,0.834203,0,...,1,0.588822,1,0.876013,1,0.5,1,0.500467,0,0.499831
3,1,49.90,9.12,0,0.001574,1,0.995625,1,0.869924,0,...,1,0.597911,1,0.81037,1,0.5,1,0.500579,0,0.499831
4,1,69.90,9.12,0,0.002352,1,0.996301,1,0.917026,0,...,1,0.540198,1,0.803598,1,0.5,1,0.500467,0,0.499831


In [53]:
funcion_costo_11.head()

,clase_test_FT_11_FN,clase_test_FT_11_FP,clase_test_FT_11_TN,clase_test_FT_11_TP,clase_test_fts_GBC_11_FN,clase_test_fts_GBC_11_FP,clase_test_fts_GBC_11_TN,clase_test_fts_GBC_11_TP,clase_test_fts_LR_11_FN,clase_test_fts_LR_11_FP,...,costo_test_ftu_RFC_11,costo_test_ftu_SVC_11,costo_test_sb_GBC_11,costo_test_sb_LR_11,costo_test_sb_RFC_11,costo_test_sb_SVC_11,costo_test_tf_GBC_11,costo_test_tf_LR_11,costo_test_tf_RFC_11,costo_test_tf_SVC_11
0,0,4610,1834,193907,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,0,1697,4747,193907,0,6444,0,193907,115,5359,...,25582.487985,25582.487985,25582.487985,-11935.619910,25582.487985,25582.487985,25582.487985,-78108.673514,25582.487985,25582.487985
2,53,1438,5006,193854,0,6444,0,193907,236,4866,...,25582.487985,25582.487985,25582.487985,-27186.077039,25582.487985,25582.487985,25582.487985,-87989.617702,25582.487985,25582.487985
3,106,1313,5131,193801,0,6444,0,193907,388,4518,...,25582.487985,25582.487985,25582.487985,-35911.125587,25582.487985,25582.487985,25582.487985,-93386.786193,25582.487985,25582.487985
4,162,1170,5274,193745,0,6444,0,193907,522,4289,...,25582.487985,25582.487985,25582.487985,-44511.880420,25582.487985,25582.487985,25582.487985,-96854.244479,25582.487985,25582.487985


In [34]:
# Experimento 12

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

feature = 'L2'
var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
x = 12

In [35]:
metricas_test_12, funcion_costo_12, best_clfs_12, best_param_12 = base_model_iter_2(train.copy(), test.copy(), x, feature, var, best_clfs, best_param, metricas_test, funcion_costo)

L2
Current date and time : 
2021-01-14 18:26:52
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L3']
---Metricas Running---
['descriptor', 'domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L3']
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L3']
---FTS----
Current date and time : 
2021-01-14 18:39:08
Current date and time : 
2021-01-14 18:39:20
----Check After Embedding----
----Check After Transform Array----
numeric features 213
categorical features 219
-----Over Sampling Done--------
Current date and time : 
2021-01-14 18:44:22
Shape X_res  (36516, 221)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-14 18:44:38
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:  8.2min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_

In [36]:
funcion_costo_12.to_pickle(r'funcion_costo_full_12.pkl')
metricas_test_12.to_pickle(r'metricas_test_full_12.pkl')

f = gzip.open('best_clfs_12.pklz','wb')
pickle.dump(best_clfs_12,f)
f.close()

f = gzip.open('best_param_12.pklz','wb')
pickle.dump(best_param_12,f)
f.close()

In [55]:
metricas_test_12.head()

,gt,price,costo_envio,pred_test_FT_12,prob_test_FT_12,pred_test_fts_LR_12,prob_test_fts_LR_12,pred_test_fts_SVC_12,prob_test_fts_SVC_12,pred_test_fts_RFC_12,...,pred_test_tf_GBC_12,prob_test_tf_GBC_12,pred_test_sb_LR_12,prob_test_sb_LR_12,pred_test_sb_SVC_12,prob_test_sb_SVC_12,pred_test_sb_RFC_12,prob_test_sb_RFC_12,pred_test_sb_GBC_12,prob_test_sb_GBC_12
0,1,120.90,14.44,0,0.000010,1,0.936022,1,0.518311,0,...,0,0.264892,1,0.957852,0,0.407477,0,0.499856,1,0.508763
1,1,270.00,11.78,0,0.000727,1,0.719892,0,0.479858,1,...,1,0.540932,1,0.847849,0,0.407055,0,0.499961,1,0.520824
2,1,200.33,14.44,0,0.000005,1,0.916057,1,0.48635,0,...,1,0.606126,1,0.92944,0,0.431058,0,0.499961,1,0.551283
3,1,49.90,9.12,0,0.000010,1,0.907386,1,0.5,1,...,1,0.593079,1,0.891064,0,0.401511,1,0.500442,1,0.504471
4,1,69.90,9.12,0,0.000010,1,0.917446,1,0.509345,1,...,0,0.493674,1,0.893227,0,0.377916,0,0.499961,0,0.467373


In [56]:
funcion_costo_12.head()

,clase_test_FT_12_FN,clase_test_FT_12_FP,clase_test_FT_12_TN,clase_test_FT_12_TP,clase_test_fts_GBC_12_FN,clase_test_fts_GBC_12_FP,clase_test_fts_GBC_12_TN,clase_test_fts_GBC_12_TP,clase_test_fts_LR_12_FN,clase_test_fts_LR_12_FP,...,costo_test_ftu_RFC_12,costo_test_ftu_SVC_12,costo_test_sb_GBC_12,costo_test_sb_LR_12,costo_test_sb_RFC_12,costo_test_sb_SVC_12,costo_test_tf_GBC_12,costo_test_tf_LR_12,costo_test_tf_RFC_12,costo_test_tf_SVC_12
0,129,4581,1863,193778,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,190488,2948,3496,3419,0,6444,0,193907,262,4705,...,25582.487985,25582.487985,25582.487985,-81278.191969,25582.487985,25582.487985,25582.487985,-3747.138066,25582.487985,25582.487985
2,191155,2796,3648,2752,0,6444,0,193907,518,4176,...,25582.487985,25582.487985,25582.487985,-90839.837407,25582.487985,25582.487985,25582.487985,-21052.747565,25582.487985,25582.487985
3,191477,2700,3744,2430,0,6444,0,193907,764,3816,...,25582.487985,25582.487985,25582.487985,-95744.304724,25582.487985,25582.487985,25582.487985,-30863.384505,25582.487985,25582.487985
4,191693,2651,3793,2214,0,6444,0,193907,1011,3532,...,25582.487985,25582.487985,25582.487985,-96577.336475,25582.487985,25582.487985,25582.487985,-39443.305810,25582.487985,25582.487985


In [37]:
# Experimento 13

metricas_test = pd.DataFrame()
funcion_costo = pd.DataFrame()
best_clfs = []
best_param = []

feature = 'L3'
var = ['title','domain_id','category_id', 'seller_id', 'brand_name', 'L1', 'L2', 'L3']
x = 13

In [38]:
metricas_test_13, funcion_costo_13, best_clfs_13, best_param_13 = base_model_iter_2(train.copy(), test.copy(), x, feature, var, best_clfs, best_param, metricas_test, funcion_costo)

L3
Current date and time : 
2021-01-16 10:53:59
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2']
---Metricas Running---
['descriptor', 'domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2']
['domain_id', 'category_id', 'seller_id', 'brand_name', 'L1', 'L2']
---FTS----
Current date and time : 
2021-01-16 11:07:00
Current date and time : 
2021-01-16 11:07:19
----Check After Embedding----
----Check After Transform Array----
numeric features 167
categorical features 173
-----Over Sampling Done--------
Current date and time : 
2021-01-16 11:13:37
Shape X_res  (36430, 175)
-----Inicio CLF-------- LR
Current date and time : 
2021-01-16 11:13:49
LR
Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  3.8min
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed: 11.8min finished
[Pipeline] . (step 1 of 4) Processing add_sintetic_

In [39]:
funcion_costo_13.to_pickle(r'funcion_costo_full_13.pkl')
metricas_test_13.to_pickle(r'metricas_test_full_13.pkl')

f = gzip.open('best_clfs_13.pklz','wb')
pickle.dump(best_clfs_13,f)
f.close()

f = gzip.open('best_param_13.pklz','wb')
pickle.dump(best_param_13,f)
f.close()

In [58]:
metricas_test_13.head() 

,gt,price,costo_envio,pred_test_FT_13,prob_test_FT_13,pred_test_fts_LR_13,prob_test_fts_LR_13,pred_test_fts_SVC_13,prob_test_fts_SVC_13,pred_test_fts_RFC_13,...,pred_test_tf_GBC_13,prob_test_tf_GBC_13,pred_test_sb_LR_13,prob_test_sb_LR_13,pred_test_sb_SVC_13,prob_test_sb_SVC_13,pred_test_sb_RFC_13,prob_test_sb_RFC_13,pred_test_sb_GBC_13,prob_test_sb_GBC_13
0,1,120.90,14.44,0,1.001358e-05,1,0.945462,0,0.53465,1,...,0,0.499967,1,0.999172,1,0.525497,0,0.499591,0,0.316561
1,1,270.00,11.78,0,3.185391e-03,1,0.914769,1,0.636999,1,...,0,0.499967,1,0.998375,0,0.408607,1,0.500536,1,0.608296
2,1,200.33,14.44,0,5.960464e-07,1,0.929592,1,0.631094,1,...,0,0.499967,1,0.99643,0,0.476935,1,0.501075,1,0.696705
3,1,49.90,9.12,0,9.894371e-06,1,0.948876,1,0.656331,1,...,0,0.499967,1,0.998303,1,0.523006,0,0.498843,0,0.437828
4,1,69.90,9.12,0,1.001358e-05,1,0.964245,1,0.692078,1,...,0,0.499967,1,0.989027,0,0.452798,1,0.501012,1,0.610895


In [59]:
funcion_costo_13.head()

,clase_test_FT_13_FN,clase_test_FT_13_FP,clase_test_FT_13_TN,clase_test_FT_13_TP,clase_test_fts_GBC_13_FN,clase_test_fts_GBC_13_FP,clase_test_fts_GBC_13_TN,clase_test_fts_GBC_13_TP,clase_test_fts_LR_13_FN,clase_test_fts_LR_13_FP,...,costo_test_ftu_RFC_13,costo_test_ftu_SVC_13,costo_test_sb_GBC_13,costo_test_sb_LR_13,costo_test_sb_RFC_13,costo_test_sb_SVC_13,costo_test_tf_GBC_13,costo_test_tf_LR_13,costo_test_tf_RFC_13,costo_test_tf_SVC_13
0,42,6424,20,193865,0,6444,0,193907,0,6444,...,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985,25582.487985
1,157269,4922,1522,36638,0,6444,0,193907,1211,3363,...,25582.487985,25582.487985,25582.487985,-82059.642094,25582.487985,25582.487985,25582.487985,-78998.574002,25582.487985,25582.487985
2,172898,4433,2011,21009,0,6444,0,193907,1995,2854,...,25582.487985,25582.487985,25582.487985,-90924.819956,25582.487985,25582.487985,25582.487985,-88480.086972,25582.487985,25582.487985
3,179296,4191,2253,14611,0,6444,0,193907,2676,2560,...,25582.487985,25582.487985,25582.487985,-95216.679846,25582.487985,25582.487985,25582.487985,-93926.656357,25582.487985,25582.487985
4,182762,3991,2453,11145,0,6444,0,193907,3221,2367,...,25582.487985,25582.487985,25582.487985,-96477.342160,25582.487985,25582.487985,25582.487985,-97142.987415,25582.487985,25582.487985
